## [문제 링크](https://www.acmicpc.net/problem/2618)
- 경찰차 2대가 최초에 (1,1)과 (N,N)에 있다
- 어떤 위치를 주면 둘 중 가까운 차가 움직인다
1. 최종적으로 이동한 최소 거리를 출력하고
2. 각 사건에 대해 어떤 차(1 or 2)가 이동했는지 출력하시오
--- 
- 어떤 이동에서 최적화된 값이 나올지 모르기 때문에 모든 방향으로 탐색을 해야 한다.
- 단, 모든 경우를 탐색하진 않는다 : 1번씩 움직였다면 최소 이동 거리가 나오는 케이스만 정리한다
- 2차원으로 DP를 구성함
    - DP의 각 row는 차 1이 이동한 횟수, 각 col은 차 2가 이동한 횟수이다
    - 각 값은 `값, 차1의 현재위치, 차2의 현재위치`로 구성해본다

## 백준에 있는 반례 다 맞는데 4%에서 틀렸습니다 뜸
- 뭐가 틀렸는지 모르겠다 gg

In [31]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

N = int(input())
W = int(input())
cases = [(0, 0)] # 케이스는 1번부터 고려한다
for _ in range(W):
    cases.append(tuple(map(int, input().split())))

    
# dp의 각 값은 (누적 거리, 1번차 위치, 2번차 위치)
dp = [[(0, 0, 0) for _ in range(W + 1)] for _ in range(W + 1)]

for i in range(1, W + 1):
    case_row = cases[i][0]
    case_col = cases[i][1]
    
    for j in range(i + 1):
        target_row = j
        target_col = i - j
        
        # dp[j][i-j]에 대해 왼쪽과 위의 값을 얻음
        
        # 1번 차의 이동 & 최초 위치 (1,1)
        if target_row == 1:
            dist1 = abs(case_row - 1) + abs(case_col - 1)
        elif target_row > 1:
            before_loc_idx_row = dp[j-1][i-j][1]
            before_loc_row = cases[before_loc_idx_row]
            dist1 = abs(case_row - before_loc_row[0]) + abs(case_col - before_loc_row[1])
        else:
            dist1 = int(1e9)

        # 2번 차의 이동
        if target_col == 1:
            dist2 = abs(case_row - N) + abs(case_col - N)
        elif target_col > 1:
            before_loc_idx_col = dp[j][i-j-1][2]
            before_loc_col = cases[before_loc_idx_col]
            dist2 = abs(case_row - before_loc_col[0]) + abs(case_col - before_loc_col[1])
        else:
            dist2 = int(1e9)
        
        print(dist1, dist2)
        total_dist1 = dp[j-1][i-j][0] + dist1
        total_dist2 = dp[j][i-j-1][0] + dist2
        
        if total_dist1 > total_dist2:
            dp[j][i-j] = (total_dist2, dp[j][i-j-1][1], i) # 2번이 움직였다면 2번의 현재 위치 정보를 갱신
        else:
            dp[j][i-j] = (total_dist1, i, dp[j-1][i-j][2]) # 1번이 움직였다면 1번의 위치 정보를 갱신

min_dist = int(1e9)
min_point = (W + 1, W + 1)
for j in range(W + 1):
    if dp[j][W-j][0] < min_dist:
        min_dist = dp[j][W-j][0]
        min_point = (j, W-j)

print(min_dist)

# 최소 경로에서 시작
x, y = min_point

# 최소 경로의 마지막 점에서 위와 왼쪽을 봤을 때,
# 왼쪽으로 이동할 경우 2번 인덱스가 다르고, 1번 인덱스가 같아야 함
# 위로 이동할 경우 1번 인덱스가 다르고, 2번 인덱스가 같아야 함
ans = []
while x != 0 or y != 0: # 둘 모두 0이 될 때까지 돌아가야 하니까, or조건이 들어간다.
    if x > 0 and dp[x-1][y][2] == dp[x][y][2] and dp[x-1][y][1] != dp[x][y][1]: # row가 바뀌고 col이 동일하다면 1번 차가 움직임
        ans.append(1) 
        x -= 1
    elif y > 0 and dp[x][y-1][1] == dp[x][y][1] and dp[x][y-1][2] != dp[x][y][2]: # row가 유지된다면 2번 차가 움직임
        ans.append(2)
        y -= 1

for i in ans[::-1]:
    print(i)

1000000000 6
2 1000000000
1000000000 4
6 2
4 1000000000
4
1
2


[(0, 0, 0), (6, 0, 1), (10, 0, 2)]
[(2, 1, 0), (4, 1, 2), (0, 0, 0)]
[(6, 2, 0), (0, 0, 0), (0, 0, 0)]


- 예시 : (28, 4, 5)에서 출발함
- 위, 왼쪽의 [1], [2]번째 인덱스를 봤을 때 왼쪽은 [1]이 4로 동일한 반면 위는 동일한 점이 없음(여기서 온 점이 아니라는 뜻)
- 따라서 왼쪽으로 이동함
- 이 과정을 x, y가 모두 0이 될 때까지 반복하면 됨

- **근데 어차피 왼쪽에서 왔다면 [2] 인덱스가 바뀌었을 거고([1]이 동일할 거고 위에서 왔다면 [1] 인덱스가 바뀌었을 거임([2]가 유지될 거임)**

In [6]:
import sys
sys.stdin = open('test.txt', 'r')
sys.setrecursionlimit(10**6)
input = sys.stdin.readline

def findDP(dp, case1, case2, car1, car2, n):
    if dp[car1][car2] != -1:
        return dp[car1][car2]
    if car1 == n or car2 == n:
        return 0
    nextCase = max(car1, car2) + 1

    car1Dis = abs(case1[car1][0] - case1[nextCase][0]) + abs(case1[car1][1] - case1[nextCase][1])
    car2Dis = abs(case2[car2][0] - case2[nextCase][0]) + abs(case2[car2][1] - case2[nextCase][1])

    result1 = car1Dis + findDP(dp, case1, case2, nextCase, car2, n)
    result2 = car2Dis + findDP(dp, case1, case2, car1, nextCase, n)

    dp[car1][car2] = min(result1, result2)
    return dp[car1][car2]

def pathPrint(dp, case1, case2, car1, car2, n):
    if car1 == n or car2 == n:
        return 0
    nextCase = max(car1, car2) + 1

    car1Dis = abs(case1[car1][0] - case1[nextCase][0]) + abs(case1[car1][1] - case1[nextCase][1])
    car2Dis = abs(case2[car2][0] - case2[nextCase][0]) + abs(case2[car2][1] - case2[nextCase][1])

    result1 = car1Dis + dp[nextCase][car2]
    result2 = car2Dis + dp[car1][nextCase]

    if result1 < result2:
        print(1)
        pathPrint(dp, case1, case2, nextCase, car2, n)
    else:
        print(2)
        pathPrint(dp, case1, case2, car1, nextCase, n)


n = int(input())
m = int(input())

dp = [[-1]*(m+2) for _ in range(m+2)]
startLocCar1 = [(1,1)]
startLocCar2 = [(n,n)]
case = [tuple(map(int, input().split())) for _ in range(m)]
case1 = startLocCar1 + case
case2 = startLocCar2 + case

print(findDP(dp, case1, case2, 0, 0, m))
pathPrint(dp, case1, case2, 0, 0, m)

8
1
1
